## Nilay Saraf 21BAI1067

# **Step 1: Install and Import Required Libraries**

---



In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# **Step 2: Load and Prepare the Dataset**

---



In [3]:
dataset = load_dataset("daily_dialog")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

daily_dialog.py:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Display a sample dialogue

In [5]:
sample_dialogue = dataset['train'][0]
print("Sample Dialogue:")
for i, utterance in enumerate(sample_dialogue['dialog']):
    print(f"Speaker {i % 2 + 1}: {utterance}")
    print(f"Emotion: {sample_dialogue['emotion'][i]}")
    print(f"Dialogue Act: {sample_dialogue['act'][i]}")
    print("---")

Sample Dialogue:
Speaker 1: Say , Jim , how about going for a few beers after dinner ? 
Emotion: 0
Dialogue Act: 3
---
Speaker 2:  You know that is tempting but is really not good for our fitness . 
Emotion: 0
Dialogue Act: 4
---
Speaker 1:  What do you mean ? It will help us to relax . 
Emotion: 0
Dialogue Act: 2
---
Speaker 2:  Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? 
Emotion: 0
Dialogue Act: 2
---
Speaker 1:  I guess you are right.But what shall we do ? I don't feel like sitting at home . 
Emotion: 0
Dialogue Act: 2
---
Speaker 2:  I suggest a walk over to the gym where we can play singsong and meet some of our friends . 
Emotion: 0
Dialogue Act: 3
---
Speaker 1:  That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . 
Emotion: 4
Dialogue Act: 4
---
Speaker 2:  Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exer

# **Step 3: Sentiment Analysis with DistilBERT**

---



## Load the DistilBERT model and tokenizer

In [6]:
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased',clean_up_tokenization_spaces=True)
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Function to get sentiment from DistilBERT

In [7]:
def get_sentiment(text):
    inputs = distilbert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = distilbert_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment = torch.argmax(probs, dim=-1).item()
    return sentiment

## Example: Get sentiment for a dialogue

In [8]:
dialogue_text = " ".join(sample_dialogue['dialog'])
sentiment = get_sentiment(dialogue_text)
print(f"Dialogue  :{dialogue_text}")
print(f"Sentiment :{sentiment}")

Dialogue  :Say , Jim , how about going for a few beers after dinner ?   You know that is tempting but is really not good for our fitness .   What do you mean ? It will help us to relax .   Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?   I guess you are right.But what shall we do ? I don't feel like sitting at home .   I suggest a walk over to the gym where we can play singsong and meet some of our friends .   That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .   Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .   Good.Let ' s go now .   All right . 
Sentiment :2


# **Step 4: Guided Dialogue Generation with GPT-2**

---



## Load the GPT-2 model and tokenizer

In [9]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Adjust the tokenizer to use a pad_token

In [10]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

## Tokenize the data, including sentiment

In [11]:
def tokenize_with_sentiment(examples):
    dialogues = [" ".join(utterance) for utterance in examples["dialog"]]
    sentiments = [get_sentiment(dialogue) for dialogue in dialogues]
    tokenized = gpt2_tokenizer(dialogues, padding="max_length", truncation=True, max_length=512)

    # Incorporate sentiment as an additional token
    tokenized["input_ids"] = [[sentiment] + ids for sentiment, ids in zip(sentiments, tokenized["input_ids"])]
    tokenized["attention_mask"] = [[1] + mask for mask in tokenized["attention_mask"]]
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

## Apply tokenization to the entire dataset

In [12]:
tokenized_datasets = dataset.map(tokenize_with_sentiment, batched=True, remove_columns=["dialog", "act", "emotion"])

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Define training arguments

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=200,
    remove_unused_columns=False,
)

## Define the Trainer for fine-tuning

In [17]:
trainer = Trainer(
    model=gpt2_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=gpt2_tokenizer,
)

## Train the model

In [18]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
200,0.787500
400,0.716900
600,0.667900
800,0.647700
1000,0.670700
1200,0.618700
1400,0.659000
1600,0.661700
1800,0.643600
2000,0.664700


TrainOutput(global_step=5559, training_loss=0.6513877123417368, metrics={'train_runtime': 2058.5029, 'train_samples_per_second': 5.401, 'train_steps_per_second': 2.701, 'total_flos': 2910718727424000.0, 'train_loss': 0.6513877123417368, 'epoch': 1.0})

# **Step 5: Save the Fine-Tuned Model**

---



In [19]:
model_save_path = "./sentiment_gpt2"
gpt2_model.save_pretrained(model_save_path)
gpt2_tokenizer.save_pretrained(model_save_path)

('./sentiment_gpt2/tokenizer_config.json',
 './sentiment_gpt2/special_tokens_map.json',
 './sentiment_gpt2/vocab.json',
 './sentiment_gpt2/merges.txt',
 './sentiment_gpt2/added_tokens.json')

In [21]:
import os
from google.colab import files
os.listdir()
dir = 'sentiment_gpt2'
f = os.listdir(dir)
print(f)
for i in f:
  files.download(dir+'/'+i)

['vocab.json', 'model.safetensors', 'config.json', 'special_tokens_map.json', 'generation_config.json', 'merges.txt', 'tokenizer_config.json']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Step 6: Load and Use the Saved Model**

---



## Import Libraries

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

## Load the trained GPT-2 model and tokenizer

In [2]:
model_save_path = "./sentiment_gpt2"
gpt2_model = GPT2LMHeadModel.from_pretrained(model_save_path)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(model_save_path)

## Load DistilBERT Model

In [3]:
sentiment_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_model_revision = "af0f99b"
sentiment_analyzer = pipeline("sentiment-analysis", model=sentiment_model_name, revision=sentiment_model_revision)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Function to generate dialogue based on a prompt and sentiment

In [4]:
def get_sentiment(text):
  sentiment = sentiment_analyzer(text)[0]
  sentiment = sentiment['label'].lower()
  return sentiment

In [5]:
def generate_dialogue_with_sentiment(prompt):

    sentiment = get_sentiment(prompt)
    if(sentiment == "negative"):
      sentiment=0
    else:
      sentiment=1

    input_ids = gpt2_tokenizer.encode(f"{sentiment} {prompt}", return_tensors="pt")
    output = gpt2_model.generate(input_ids, max_length=100, pad_token_id=gpt2_tokenizer.eos_token_id)
    output = gpt2_tokenizer.decode(output[0], skip_special_tokens=True)
    output = output.split("  ")[1:]

    y = list()
    for i in output:

      if(i not in y):
        y.append(i)

      else:
        break

    y = " ".join(y)
    return y

## Example: Generate a dialogue

In [31]:
g1 = generate_dialogue_with_sentiment("I had a bad day.")
print(f"Generated Dialogue : {g1}")

Generated Dialogue :  I'm sorry.


In [28]:
g2 = generate_dialogue_with_sentiment("It was raining. I am sad.")
print(f"Generated Dialogue : {g2}")

Generated Dialogue :  I am sorry.It was raining.


In [29]:
g3 = generate_dialogue_with_sentiment("You make me so proud!")
print(f"Generated Dialogue : {g3}")

Generated Dialogue :  You are so lucky!  I am so lucky too.  You are so lucky too.


In [6]:
g4 = generate_dialogue_with_sentiment("You are a wonderful friend!")
print("Generated Dialogue:", g4)

Generated Dialogue:  I am very happy to hear that.


In [8]:
g4 = generate_dialogue_with_sentiment("This project is neither good nor bad")
print("Generated Dialogue:", g4)

Generated Dialogue:  I think it's a good idea.  I agree.
